In [1]:
import os, sys
sys.path.insert(0, os.path.abspath(".."))

In [2]:
import boto3
import pickle
import numpy as np
import pandas as pd
import datetime
from datetime import datetime as dt
from dateutil.relativedelta import *

In [3]:
pd.options.display.max_columns = None

In [4]:
session = boto3.session.Session()
s3client = session.client('s3')

In [5]:
response = s3client.get_object(Bucket='inai-summerofdata', Key='mlpreproc/pickles_temporalidad/df_preproc.pkl')

body = response['Body'].read()
dum_df = pickle.loads(body)

In [6]:
response = s3client.get_object(Bucket='inai-summerofdata', Key='mlpreproc/pickles_temporalidad/index_cv_split.pkl')

body = response['Body'].read()
index_output = pickle.loads(body)

In [7]:
dum_df.head()

,folio,fechasolicitud,anosolicitud,messolicitud,semanasolicitud,diasolicitud,descripcionsolicitud_lemma,solicitud_lemma_longitud,dependencia_clean_administracion portuaria integral,dependencia_clean_administracion publica federal,dependencia_clean_aeropuertos y servicios auxiliares (asa),dependencia_clean_archivo general de la nación,dependencia_clean_auditoría superior de la federación,dependencia_clean_banca de desarrollo,dependencia_clean_camara de dipuados,dependencia_clean_capufe,dependencia_clean_cfe,dependencia_clean_cnbv,dependencia_clean_cofepris,dependencia_clean_conacyt,dependencia_clean_empresas de participacion estatal,dependencia_clean_hospitales,dependencia_clean_inah,dependencia_clean_inai,dependencia_clean_ine,dependencia_clean_instituciones de educacion superior autonomas,dependencia_clean_instituto del fondo nacional de la vivienda para los trabajadores,dependencia_clean_instituto federal de telecomunicaciones (ift),dependencia_clean_instituto mexicano de la propiedad industrial,dependencia_clean_instituto mexicano del seguro social (imss),dependencia_clean_instituto nacional de migración,dependencia_clean_ipn,dependencia_clean_lotenal,dependencia_clean_organismo autonomo,dependencia_clean_organismo descentralizado,dependencia_clean_partidos politicos,dependencia_clean_pemex,dependencia_clean_pgr,dependencia_clean_poder judicial de la federacion,dependencia_clean_policia federal,dependencia_clean_presidencia de la republica,dependencia_clean_procuraduría federal del consumidor,dependencia_clean_profeco,dependencia_clean_registro agrario nacional,dependencia_clean_sader,dependencia_clean_sae,dependencia_clean_sagarpa,dependencia_clean_sat,dependencia_clean_scjn,dependencia_clean_sct,dependencia_clean_se,dependencia_clean_secretaria de bienestar,dependencia_clean_secretariado ejecutivo del sistema nacional de seguridad pública,dependencia_clean_secretaría de cultura,dependencia_clean_sectur,dependencia_clean_sedatu,dependencia_clean_sedena,dependencia_clean_sedesol,dependencia_clean_segob,dependencia_clean_semar,dependencia_clean_semarnat,dependencia_clean_senado de la república,dependencia_clean_sener,dependencia_clean_sep,dependencia_clean_sfp,dependencia_clean_shcp,dependencia_clean_sindicatos,dependencia_clean_sre,dependencia_clean_ssa,dependencia_clean_sspc,dependencia_clean_stps,dependencia_clean_tribunales administrativos,dependencia_clean_unam,codigo_calidad_respuesta_real
0,1857500001912,2012-01-01 05:04:07,2012,1,52,6,comportamiento historico tipicos comprender es...,424,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,1857500002012,2012-01-01 05:13:16,2012,1,52,6,usar flujo efectivo pozo tipico aguar fria ela...,289,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,0001800000712,2012-01-01 09:46:39,2012,1,52,6,proporcionar magnetico relacion ocupar amparar...,143,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,0000600003712,2012-01-01 10:11:41,2012,1,52,6,proporcionar magnetico relacion ocupar amparar...,143,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
4,0001500002812,2012-01-01 10:32:15,2012,1,52,6,proporcionar magnetico relacion ocupar amparar...,188,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# Modeling

In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from pprint import pprint

In [9]:
drop_columns = ['folio', 'codigo_calidad_respuesta_real']

In [10]:
X = dum_df.drop(drop_columns, axis=1)
y = pd.DataFrame(dum_df['codigo_calidad_respuesta_real'])

In [11]:
X.head()

,fechasolicitud,anosolicitud,messolicitud,semanasolicitud,diasolicitud,descripcionsolicitud_lemma,solicitud_lemma_longitud,dependencia_clean_administracion portuaria integral,dependencia_clean_administracion publica federal,dependencia_clean_aeropuertos y servicios auxiliares (asa),dependencia_clean_archivo general de la nación,dependencia_clean_auditoría superior de la federación,dependencia_clean_banca de desarrollo,dependencia_clean_camara de dipuados,dependencia_clean_capufe,dependencia_clean_cfe,dependencia_clean_cnbv,dependencia_clean_cofepris,dependencia_clean_conacyt,dependencia_clean_empresas de participacion estatal,dependencia_clean_hospitales,dependencia_clean_inah,dependencia_clean_inai,dependencia_clean_ine,dependencia_clean_instituciones de educacion superior autonomas,dependencia_clean_instituto del fondo nacional de la vivienda para los trabajadores,dependencia_clean_instituto federal de telecomunicaciones (ift),dependencia_clean_instituto mexicano de la propiedad industrial,dependencia_clean_instituto mexicano del seguro social (imss),dependencia_clean_instituto nacional de migración,dependencia_clean_ipn,dependencia_clean_lotenal,dependencia_clean_organismo autonomo,dependencia_clean_organismo descentralizado,dependencia_clean_partidos politicos,dependencia_clean_pemex,dependencia_clean_pgr,dependencia_clean_poder judicial de la federacion,dependencia_clean_policia federal,dependencia_clean_presidencia de la republica,dependencia_clean_procuraduría federal del consumidor,dependencia_clean_profeco,dependencia_clean_registro agrario nacional,dependencia_clean_sader,dependencia_clean_sae,dependencia_clean_sagarpa,dependencia_clean_sat,dependencia_clean_scjn,dependencia_clean_sct,dependencia_clean_se,dependencia_clean_secretaria de bienestar,dependencia_clean_secretariado ejecutivo del sistema nacional de seguridad pública,dependencia_clean_secretaría de cultura,dependencia_clean_sectur,dependencia_clean_sedatu,dependencia_clean_sedena,dependencia_clean_sedesol,dependencia_clean_segob,dependencia_clean_semar,dependencia_clean_semarnat,dependencia_clean_senado de la república,dependencia_clean_sener,dependencia_clean_sep,dependencia_clean_sfp,dependencia_clean_shcp,dependencia_clean_sindicatos,dependencia_clean_sre,dependencia_clean_ssa,dependencia_clean_sspc,dependencia_clean_stps,dependencia_clean_tribunales administrativos,dependencia_clean_unam
0,2012-01-01 05:04:07,2012,1,52,6,comportamiento historico tipicos comprender es...,424,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2012-01-01 05:13:16,2012,1,52,6,usar flujo efectivo pozo tipico aguar fria ela...,289,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2012-01-01 09:46:39,2012,1,52,6,proporcionar magnetico relacion ocupar amparar...,143,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,2012-01-01 10:11:41,2012,1,52,6,proporcionar magnetico relacion ocupar amparar...,143,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,2012-01-01 10:32:15,2012,1,52,6,proporcionar magnetico relacion ocupar amparar...,188,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [12]:
y.head()

,codigo_calidad_respuesta_real
0,1
1,1
2,0
3,1
4,0


In [13]:
features_cv = np.array(X.drop('fechasolicitud', axis=1))
labels_cv = np.array(y)
labels_cv = np.ravel(labels_cv)

In [14]:
ngram_range = (1, 2)
min_df = 10
max_df = 1.
max_features = 300

In [15]:
preprocess = ColumnTransformer(
    [('descripcionsolicitud_tfidf', TfidfVectorizer(encoding='utf-8',
                                        ngram_range=ngram_range,
                                        stop_words=None,
                                        lowercase=False,
                                        max_df=max_df,
                                        min_df=min_df,
                                        max_features=max_features,
                                        norm='l2',
                                        sublinear_tf=True), 4)],
    remainder='passthrough')

In [16]:
clf = GradientBoostingClassifier(random_state=8)

In [17]:
gbc = Pipeline([('preprocess', preprocess), ('clf', clf)])

In [18]:
max_depth = [5, 10, 15]
max_features = ['auto']
min_samples_leaf = [1, 5, 7, 9]
min_samples_split = [10, 20]
n_estimators = [800, 1000, 1300]
learning_rate = [.1, .5]
subsample = [.5]

param_grid = {
    'clf__max_depth': max_depth,
    'clf__max_features': max_features,
    'clf__min_samples_leaf': min_samples_leaf,
    'clf__min_samples_split': min_samples_split,
    'clf__n_estimators': n_estimators,
    'clf__learning_rate': learning_rate,
    'clf__subsample': subsample
}

pprint(param_grid)

{'clf__learning_rate': [0.1, 0.5],
 'clf__max_depth': [5, 10, 15],
 'clf__max_features': ['auto'],
 'clf__min_samples_leaf': [1, 5, 7, 9],
 'clf__min_samples_split': [10, 20],
 'clf__n_estimators': [800, 1000, 1300],
 'clf__subsample': [0.5]}


In [ ]:
grid_search = GridSearchCV(estimator=gbc,
                          param_grid=param_grid,
                          scoring='accuracy',
                          cv=index_output,
                          n_jobs=-1,
                          verbose=1)

grid_search.fit(features_cv , labels_cv)

In [ ]:
print(grid_search.best_params_)

In [ ]:
print(grid_search.best_score_)

In [ ]:
pd.DataFrame(grid_search.cv_results_)

Guardamos en S3

In [ ]:
bucket='inai-summerofdata'
key='modeling/GBC/1_temporalidad/2_iteracion/grid_search.pkl'
pickle_byte_obj = pickle.dumps(grid_search)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, key).put(Body=pickle_byte_obj)

**Mejor modelo**

In [ ]:
best_gbc = grid_search.best_estimator_

In [ ]:
best_gbc

##  Fit y desempeño del modelo

In [ ]:
training_accuracy = []
test_accuracy = []
for train_index, test_index in index_output:
    
    features_train = np.array(X.loc[train_index].drop('fechasolicitud', axis=1))
    labels_train = np.ravel(y.loc[train_index])
    
    features_test = np.array(X.loc[test_index].drop('fechasolicitud', axis=1))
    labels_test = np.ravel(y.loc[test_index])
    
    # Modeling train data
    best_gbc.fit(features_train, labels_train)
    
    # Predictions
    gbc_pred = best_gbc.predict(features_test)
    
    # Training accuracy
    training_accuracy.append(accuracy_score(labels_train, best_gbc.predict(features_train)))
    
    # Test accuracy
    test_accuracy.append(accuracy_score(labels_test, gbc_pred))
    
    

In [ ]:
average_training_accuracy = np.mean(training_accuracy)
average_test_accuracy = np.mean(test_accuracy)

**Average training accuracy**

In [ ]:
average_training_accuracy

**Average test accuracy**

In [ ]:
average_test_accuracy

Creamos un dataframe

In [ ]:
data = {
    'Modelo': 'GBC',
    'Average Training Set Accuracy': average_training_accuracy,
    'Average Test Set Accuracy': average_test_accuracy
}
df_models_gbc = pd.DataFrame(data, index=[0])

In [ ]:
df_models_gbc

Guardamos en S3

In [ ]:
features = np.array(X.drop('fechasolicitud', axis=1))
labels = np.ravel(y)

In [ ]:
bucket='inai-summerofdata'
key='modeling/GBC/1_temporalidad/2_iteracion/features.pkl'
pickle_byte_obj = pickle.dumps(features)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, key).put(Body=pickle_byte_obj)

In [ ]:
bucket='inai-summerofdata'
key='modeling/GBC/1_temporalidad/2_iteracion/labels.pkl'
pickle_byte_obj = pickle.dumps(labels)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, key).put(Body=pickle_byte_obj)

In [ ]:
bucket='inai-summerofdata'
key='modeling/GBC/1_temporalidad/2_iteracion/df_models_gbc.pkl'
pickle_byte_obj = pickle.dumps(df_models_gbc)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, key).put(Body=pickle_byte_obj)

In [ ]:
bucket='inai-summerofdata'
key='modeling/GBC/1_temporalidad/2_iteracion/best_gbc.pkl'
pickle_byte_obj = pickle.dumps(best_gbc)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, key).put(Body=pickle_byte_obj)